# Instituto Tecnológico y de Estudios Superiores de Monterrey
## Maestría en Inteligencia Artificial Aplicada
### Proyecto Integrador (Gpo 10) - TC5035.10

### **Proyecto: Diseño Acelerado de Fármacos**

### Avance 2: Ingeniería de características

#### **Docentes:**
- Dra. Grettel Barceló Alonso - Profesor Titular
- Dr. Luis Eduardo Falcón Morales - Profesor Titular
- Dr. Horacio Martinez Alfaro  – Profesor Tutor

#### **Miembros del equipo:**
 - Jose Luis Artigas Arroyo - A01794906
 - José Luis Fabela Portillo - A00748551
 - Alejandro Emmanuel Silva Ortega - A01794545

#### Para agregrar descripcion en caso de requerir

In [1]:
# Importar librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re


In [2]:
#leer csv con dataset preprocesado
df_peptidos = pd.read_csv('../data/processed/peptide_processed.csv')
df_peptidos.head(3).T

,0,1,2
Sequence,RGLRRLGRKIAHGVKKYGPTVLRIIRIA,KWKLFKKIGIGKFLHSAKKF,KWKLFKKIGPGKFLHSAKKF
Sequence_Length,28,20,20
Origin,Ovis aries,Ceropin-Magainin hybrid peptides,Ceropin-Magainin hybrid peptides
Type,Native peptide,Synthetic peptide,Synthetic peptide
Hemolytic_Activity,Human erythrocytes (4% Hemolysis=4 µM; 19% Hem...,Human erythrocytes (Hemolysis 0% =100 μM),Human erythrocytes: Hemolysis 0% (100 μM)
Cytotoxicity,NIH 3T3: IC50=16 µM,NIH-3T3 fibroblast: IC50>100 μM,NIH-3T3 fibroblast: IC50>100 μM
Binding_Target,NaN,NaN,NaN
Affinity,NaN,NaN,NaN
Mechanism,NaN,NaN,NaN
Nature,Anticancer; Antibacterial; Antifungal,Anticancer; Antibacterial,Anticancer; Antibacterial


## Analisis de secuencias de péptidos

De acuerdo a la IUPAC[1], los aminoacidos se clasifican de acuerdo a la siguiente tabla


|symbol|Amino acid|
| :-:|:-:|
|A	| alanine|
|B	| aspartic acid or asparagine|
|C	| cysteine|
|D	| aspartic acid|
|E	| glutamic acid|
|F	| phenylalanine|
|G	| glycine|
|H	| histidine|
|I	| isoleucine|
|K	| lysine|
|L	| leucine|
|M	| methionine|
|N	| asparagine|
|P	| proline|
|Q	| glutamine|
|R	| arginine|
|S	| serine|
|T	| threonine|
|U  | selenocysteine|
|V	| valine|
|W	| tryptophan|
|X  | Desconocido u 'otro'|
|Y	| tyrosine|
|Z	| glutamic acid or glutamine|

Por lo cual, se eliminaran de la secuencia los caractares diferentes a los indicados en esta lista, de igual forma se actualizara la longitud de la secuencia para hacerla coincidir con la longitud despues de eliminar los caracteres invalidos

In [3]:
df_peptidos["Sequence"]

0                RGLRRLGRKIAHGVKKYGPTVLRIIRIA
1                        KWKLFKKIGIGKFLHSAKKF
2                        KWKLFKKIGPGKFLHSAKKF
3                           KWKLFKKIKFLHSAKKF
4                          KWKLFKKIPKFLHSAKKF
                        ...                  
5846                 FLPLLAGLAANFLPKIICKIARKC
5847                  RIIDRLWLVRRPOKPKFVLVWVL
5848                               FHAVPQSFYT
5849                            KLWCKSSQVPQSR
5850    TKEQKEQIAKATGLTTKQVRNWYVQLNASIKVCMCSC
Name: Sequence, Length: 5851, dtype: object

### Eliminacion de columnas redundantes
Despues de un analis mas profundo, y una revision con el experto en el tema, se procedera a la eliminacion de columnas redundantes, ya que estos valores pueden ser obtenidos direcvtamente de la secuencia de peptidos, estas columnas son

- Chiral: Los aminoácidos L/D consisten en péptidos.
- Mass: Masa
- Formula: Formula del péptido
- Absent_amino_acids: Aminoacidos no incluidos
- Common_amino_acids: Animoacido mas comun en la sequencia
- pI: Punto isoeléctrico
- Basic_residues: Número de residuos base
- Acidic_residues: Número de residuos ácidos
- Net_charge: Carga Neta
- Polar_residues: Número de residuos polares
- Hydrophobic_residues: Número de residuos hidrofóbicos
- Hydrophobicity: Hidrofobicidad
- Boman_Index: Índice de Boman
- Half_Life: Vida media
- Aliphatic_Index: Índice alifático
- Extinction_Coefficient_cystines: Coeficiente de extinción de cistinas
- Absorbance_280nm: Absorbencia a 280nm


In [4]:
drop_cols = ['Chiral','Mass','Formula','Absent_amino_acids','Common_amino_acids','pI','Basic_residues','Acidic_residues','Net_charge','Polar_residues','Hydrophobic_residues','Hydrophobicity','Boman_Index',
             'Half_Life','Aliphatic_Index','Extinction_Coefficient_cystines','Absorbance_280nm']
df_peptidos.drop(drop_cols, axis=1, inplace=True)

In [5]:
df_peptidos.head(5).T

,0,1,2,3,4
Sequence,RGLRRLGRKIAHGVKKYGPTVLRIIRIA,KWKLFKKIGIGKFLHSAKKF,KWKLFKKIGPGKFLHSAKKF,KWKLFKKIKFLHSAKKF,KWKLFKKIPKFLHSAKKF
Sequence_Length,28,20,20,17,18
Origin,Ovis aries,Ceropin-Magainin hybrid peptides,Ceropin-Magainin hybrid peptides,Ceropin-Magainin hybrid peptides,Ceropin-Magainin hybrid peptides
Type,Native peptide,Synthetic peptide,Synthetic peptide,Synthetic peptide,Synthetic peptide
Hemolytic_Activity,Human erythrocytes (4% Hemolysis=4 µM; 19% Hem...,Human erythrocytes (Hemolysis 0% =100 μM),Human erythrocytes: Hemolysis 0% (100 μM),Human erythrocytes: Hemolysis 0% (100 μM),Human erythrocytes: Hemolysis 0% (100 μM)
Cytotoxicity,NIH 3T3: IC50=16 µM,NIH-3T3 fibroblast: IC50>100 μM,NIH-3T3 fibroblast: IC50>100 μM,NIH-3T3 fibroblast: IC50>100 μM,NIH-3T3 fibroblast: IC50>100 μM
Binding_Target,NaN,NaN,NaN,NaN,NaN
Affinity,NaN,NaN,NaN,NaN,NaN
Mechanism,NaN,NaN,NaN,NaN,NaN
Nature,Anticancer; Antibacterial; Antifungal,Anticancer; Antibacterial,Anticancer; Antibacterial,Anticancer; Antibacterial,Anticancer; Antibacterial


### Limpieza de datos
se eliminaran los registros con cadenas que contengan "-" y numeros (1-9) ya que estas representan peptidos modificados y contienen compuestos quimicos los cuales modifican el péptido y estos no seran considerados para el modelo, de igual forma se eliminaran las secuencias que contengan el caracter Ⓧ ya que este caracter representa peptidos ciclicos los cuales no se tomaran en cuenta para este modelo ya que no representan una muestra significativa.

In [ ]:
# Aminoácidos IUPAC
aminoacidos = ['A','B','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','U','V','W','X','Y','Z']

# Crear un diccionario para contar la aparición de cada carácter
char_count = {}

# Iterar sobre cada secuencia en la columna 'Sequence'
for sequence in df_peptidos['Sequence']:
    # Convertir la secuencia en un conjunto de caracteres únicos
    unique_chars = set(sequence)
    # Contar la aparición de cada carácter en el conjunto
    for char in unique_chars:
        if char in char_count:
            char_count[char] += 1
        else:
            char_count[char] = 1

# Convertir el diccionario en un DataFrame para una mejor visualización
char_count_df = pd.DataFrame(list(char_count.items()), columns=['Character', 'Count'])
char_count_df['%'] = (char_count_df['Count'] / df_peptidos.shape[0]).round(4) *100

print('Total de caracteres únicos: ', len(char_count_df))
print('Total de aminoácidos IUPAC: ', len(aminoacidos))
print('Total de caracteres no IUPAC: ', len(char_count_df)-len(aminoacidos))

Total de péptidos:  5851
Total de caracteres únicos:  64
Total de aminoácidos IUPAC:  24
Total de caracteres no IUPAC:  40


In [31]:
# Número de péptidos en los que está cada aminoácido IUPAC:
char_count_df[char_count_df['Character'].isin(aminoacidos)].sort_values(by='Count', ascending=False)

,Character,Count,%
0,L,3858,65.94
3,K,3390,57.94
4,R,3207,54.81
1,I,3081,52.66
5,A,3014,51.51
6,G,2964,50.66
10,V,2932,50.11
11,F,2875,49.14
7,P,2435,41.62
13,W,2143,36.63


In [32]:
# Número de péptidos en los que está cada caracter que no es aminoácido IUPAC:
char_count_df[~char_count_df['Character'].isin(aminoacidos)].sort_values(by='Count', ascending=False)

,Character,Count,%
46,Ⓧ,236,4.03
30,r,141,2.41
23,l,132,2.26
25,k,112,1.91
22,f,78,1.33
39,x,77,1.32
28,a,54,0.92
21,v,54,0.92
31,w,52,0.89
24,i,44,0.75


In [42]:
# crea un dataframe que contenga solo los registros con caracteres de aminoacidos en la columna 'sequence'
filtered_df = df_peptidos[df_peptidos['Sequence'].apply(lambda x: all(char in aminoacidos for char in x))]
print('Total de péptidos: ', df_peptidos.shape[0])
print('Péptidos solo con aminoácidos UIPAC: ', filtered_df.shape[0])
print('Péptidos eliminados: ', round((df_peptidos.shape[0]-filtered_df.shape[0])/df_peptidos.shape[0],4) * 100, '%')

Total de péptidos:  5851
Péptidos solo con aminoácidos UIPAC:  5124
Péptidos eliminados:  12.43 %


In [43]:
filtered_df["Sequence"]

0                RGLRRLGRKIAHGVKKYGPTVLRIIRIA
1                        KWKLFKKIGIGKFLHSAKKF
2                        KWKLFKKIGPGKFLHSAKKF
3                           KWKLFKKIKFLHSAKKF
4                          KWKLFKKIPKFLHSAKKF
                        ...                  
5845                 FLPLLAGLAANFLPQIICKIARKC
5846                 FLPLLAGLAANFLPKIICKIARKC
5848                               FHAVPQSFYT
5849                            KLWCKSSQVPQSR
5850    TKEQKEQIAKATGLTTKQVRNWYVQLNASIKVCMCSC
Name: Sequence, Length: 5124, dtype: object

### Separacion de tipo de de aminoácidos
Se separaron los aminoácidos entre aminoácidos naturales y sinteticos, estos aminoácidos pueden estar mezclados en la secuencia, y se distinguen entre ellos en la secuencia con caracteres en mayusculas para los aminoácidos tipo D o naturales y minusculas para los aminoácidos L o sintéticos


In [14]:
df_peptidos["D_amino"] = df_peptidos["Sequence"].map(lambda x:re.sub(r'[^A-Z]','',x)).replace('',None)
df_peptidos["L_amino"] = df_peptidos["Sequence"].map(lambda x:re.sub(r'[^a-z]','',x)).replace('',None)

In [15]:
print(f'Porcentaje de L aminoácidos nulos: {(df_peptidos["L_amino"].isnull().sum()/df_peptidos.shape[0]).round(2)}')
print(f'Porcentaje de D aminoácidos nulos: {(df_peptidos["D_amino"].isnull().sum()/df_peptidos.shape[0]).round(2)}')

Porcentaje de L aminoácidos nulos: 0.92
Porcentaje de D aminoácidos nulos: 0.01


In [16]:
df_peptidos["Sequence"] = df_peptidos["Sequence"].map(lambda x:re.sub(r'[^ABCDEFGHIKLMNPQRSTUVWXYZ]','',x).upper())
df_peptidos["Sequence"]

0                RGLRRLGRKIAHGVKKYGPTVLRIIRIA
1                        KWKLFKKIGIGKFLHSAKKF
2                        KWKLFKKIGPGKFLHSAKKF
3                           KWKLFKKIKFLHSAKKF
4                          KWKLFKKIPKFLHSAKKF
                        ...                  
5846                 FLPLLAGLAANFLPKIICKIARKC
5847                   RIIDRLWLVRRPKPKFVLVWVL
5848                               FHAVPQSFYT
5849                            KLWCKSSQVPQSR
5850    TKEQKEQIAKATGLTTKQVRNWYVQLNASIKVCMCSC
Name: Sequence, Length: 5851, dtype: object

### Convertir dataframe to FASTA 

In [17]:
#df_fasta = list()
with open('../data/processed/VEGF.fasta','w', encoding='UTF8') as f:
    for _, row in df_peptidos.iterrows():
        f.write(f">a{row['VEGF']}\n{row['Sequence']}\n")
    


In [18]:
#Ejecutar cd-hit en colab, para windows usar el comando con wsl
#! cd-hit -i data/processed/VEGF.fasta -o data/processed/VEGF.txt -c 0.99

import subprocess

# Define the WSL command buscar 
command = ["wsl", "cd-hit", "-i", "../data/processed/VEGF.fasta", "-o", "../data/processed/VEGF.txt", "-c", "0.99"]

# Run the command
result = subprocess.run(command, capture_output=True, text=True)

# Print the output and errors
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)


STDOUT: ================================================================
Program: CD-HIT, V4.8.1 (+OpenMP), Aug 20 2021, 08:39:56
Command: cd-hit -i ../data/processed/VEGF.fasta -o
         ../data/processed/VEGF.txt -c 0.99

Started: Wed Jan 29 18:46:48 2025
                            Output                              
----------------------------------------------------------------
total seq: 4158
longest and shortest : 97 and 11
Total letters: 86698
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 1 X 10M = 10M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 76M

Table limit with the given memory limit:
Max number of representatives: 4000000
Max number of word counting entries: 90425000


comparing sequences from          0  to       4158
....
     4158  finished       3605  clusters

Approximated maximum memory consumption: 77M
writing new database
writing clustering information
program completed !

### Pfeature test

In [19]:
from pfeature import pcp_wp

In [20]:
# Dipeptide composition (PCP)

def pcp(input):
  a = input.rstrip('txt')
  output = a + 'dpc.csv'
  df_out = pcp_wp(input, output)
  df_in = pd.read_csv(output)
  return df_in

feature_pcp = pcp('../data/processed/VEGF.txt')
feature_pcp

AttributeError: 'float' object has no attribute 'upper'

### Referencias

1. INTERNATIONAL UNION OF PURE AND APPLIED CHEMISTRY (IUPAC). https://iupac.qmul.ac.uk/AminoAcid/A2021.html#AA20